# Answers

## A1 – Connect and Load Data

_Connect to your local DuckDB database and load the `penguins` table into a pandas dataframe using Python._

In [ ]:
import duckdb
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

# Read the DuckDB database file from .env file
DB_PATH = os.getenv("DUCKDB_DATABASE_PATH")

# update with the path to your DuckDB database file
con = duckdb.connect(database=DB_PATH, read_only=True)

penguins_df = con.execute(
    "SELECT * FROM penguins;"
).df()
penguins_df.head()
# Preview the first rows of the penguins dataset now loaded in pandas.


## A2 – Count Penguins by Species

_Query `penguins` to list each species alongside its row count, ordered from most to fewest records._

In [ ]:
con.execute(
    """
    SELECT
        species,
        COUNT(*) AS penguin_count
    FROM penguins
    GROUP BY species
    ORDER BY penguin_count DESC;
    """
).df()
# Adelie typically appears with the highest count.


## A3 – Average Body Mass by Island

_Calculate the average `body_mass_g` for penguins on each island and round the result to the nearest gram._

In [ ]:
con.execute(
    """
    SELECT
        island,
        ROUND(AVG(body_mass_g)) AS avg_body_mass_g
    FROM penguins
    GROUP BY island
    ORDER BY island;
    """
).df()
# Shows how average body mass differs by island habitat.


## A4 – Top Heaviest Penguins

_Return the 10 heaviest penguins with their species, sex, island, and `body_mass_g`, ordered by mass descending._

In [ ]:
con.execute(
    """
    SELECT
        species,
        sex,
        island,
        body_mass_g
    FROM penguins
    WHERE body_mass_g IS NOT NULL
    ORDER BY body_mass_g DESC
    LIMIT 10;
    """
).df()
# Highlights the individuals with the greatest recorded body mass.


## A5 – Bill Metrics by Species and Sex

_Compute the average `bill_length_mm` and `bill_depth_mm` for each combination of species and sex, excluding rows where sex is NULL._

In [ ]:
con.execute(
    """
    SELECT
        species,
        sex,
        ROUND(AVG(bill_length_mm), 2) AS avg_bill_length_mm,
        ROUND(AVG(bill_depth_mm), 2) AS avg_bill_depth_mm
    FROM penguins
    WHERE sex IS NOT NULL
    GROUP BY species, sex
    ORDER BY species, sex;
    """
).df()
# Provides side-by-side bill metrics by species-sex group.


## A6 – Flipper Length Change by Year

_For each species, calculate the average `flipper_length_mm` by year and add a column showing the change from the previous year._

In [ ]:
con.close()

In [ ]:
con.execute(
    """
    WITH avg_flipper AS (
        SELECT
            species,
            island,
            AVG(flipper_length_mm) AS avg_flipper_length_mm
        FROM penguins
        GROUP BY species, island
    )
    SELECT
        species,
        island,
        ROUND(avg_flipper_length_mm, 2) AS avg_flipper_length_mm,
        ROUND(
            avg_flipper_length_mm - LAG(avg_flipper_length_mm) OVER (
                PARTITION BY species ORDER BY island
            ),
            2
        ) AS change_from_prior_island
    FROM avg_flipper
    ORDER BY species, island;
    """
).df()
# Shows average flipper length and change across islands for each species.


## A7 – Missing Body Mass Share

_For each species, compute the percentage of records with NULL `body_mass_g`._

In [ ]:
con.execute(
    """
    SELECT
        species,
        ROUND(
            100.0 * SUM(CASE WHEN body_mass_g IS NULL THEN 1 ELSE 0 END) / COUNT(*),
            2
        ) AS pct_missing_body_mass
    FROM penguins
    GROUP BY species
    ORDER BY species;
    """
).df()
# Reveals how prevalent missing body mass measurements are per species.


## A8 – Average Tip Percentage by Day and Time

_Using the `tips` table, calculate the average tip percentage (`tip / total_bill`) for each combination of `day` and `time`._

In [ ]:
con.execute(
    """
    SELECT
        day,
        time,
        ROUND(AVG(tip / total_bill) * 100, 2) AS avg_tip_pct
    FROM tips
    GROUP BY day, time
    ORDER BY day, time;
    """
).df()
# Displays tipping tendencies across day/time combinations.


## A9 – Top Tip Percentages

_Return the five rows with the highest tip percentage, showing `total_bill`, `tip`, `size`, and the calculated percentage. Break ties by higher `total_bill`._

In [ ]:
con.execute(
    """
    SELECT
        total_bill,
        tip,
        size,
        ROUND(tip / total_bill * 100, 2) AS tip_pct
    FROM tips
    ORDER BY tip_pct DESC, total_bill DESC
    LIMIT 5;
    """
).df()
# Captures the most generous tipping instances, favoring larger bills on ties.


## A10 – Smoker and Sex Comparison

_Compute the average tip percentage for each grouping of `smoker` status and `sex`, including the number of meals in each group._

In [ ]:
con.execute(
    """
    SELECT
        smoker,
        sex,
        COUNT(*) AS meal_count,
        ROUND(AVG(tip / total_bill) * 100, 2) AS avg_tip_pct
    FROM tips
    GROUP BY smoker, sex
    ORDER BY smoker, sex;
    """
).df()
# Helps compare tipping behavior across smoker/sex combinations.


## A11 – Party Size Revenue Summary

_Summarize the `tips` data by party `size`, returning the count of bills, total `total_bill`, average `tip`, and average tip percentage._

In [ ]:
con.execute(
    """
    SELECT
        size,
        COUNT(*) AS bill_count,
        ROUND(SUM(total_bill), 2) AS total_revenue,
        ROUND(AVG(tip), 2) AS avg_tip,
        ROUND(AVG(tip / total_bill) * 100, 2) AS avg_tip_pct
    FROM tips
    GROUP BY size
    ORDER BY size;
    """
).df()
# Summarizes revenue and generosity patterns by table size.


## A12 – Highest Median Total Bill Day

_Calculate the median `total_bill` for each `day` and identify the day(s) with the highest median._

In [ ]:
con.execute(
    """
    WITH day_medians AS (
        SELECT
            day,
            MEDIAN(total_bill) AS median_total_bill
        FROM tips
        GROUP BY day
    )
    SELECT
        day,
        ROUND(median_total_bill, 2) AS median_total_bill,
        CASE
            WHEN median_total_bill = (SELECT MAX(median_total_bill) FROM day_medians)
            THEN TRUE
            ELSE FALSE
        END AS is_highest_median
    FROM day_medians
    ORDER BY median_total_bill DESC;
    """
).df()
# Flags the day(s) with the top median spending.


## A13 – Rank Tip Percentage Within Day

_Within each `day`, rank meals by tip percentage using `DENSE_RANK`, retaining `total_bill`, `tip`, and the computed rank._

In [ ]:
con.execute(
    """
    WITH ranked AS (
        SELECT
            day,
            total_bill,
            tip,
            ROUND(tip / total_bill * 100, 2) AS tip_pct,
            DENSE_RANK() OVER (
                PARTITION BY day
                ORDER BY tip / total_bill DESC
            ) AS tip_pct_rank
        FROM tips
    )
    SELECT
        day,
        total_bill,
        tip,
        tip_pct,
        tip_pct_rank
    FROM ranked
    ORDER BY day, tip_pct_rank, tip_pct DESC;
    """
).df()
# Produces within-day rankings for tip generosity.


## A14 – Overall Survival Rate

_From the `titanic` table, compute the overall survival rate as the percentage of passengers where `survived = 1`._

In [ ]:
con.execute(
    """
    SELECT
        ROUND(AVG(CAST(survived AS DOUBLE)) * 100, 2) AS survival_rate_pct
    FROM titanic;
    """
).df()
# Returns the overall survival percentage across all passengers.


## A15 – Survival by Class and Sex

_Calculate the survival rate for each combination of passenger class (`pclass`) and `sex`._

In [ ]:
con.execute(
    """
    SELECT
        pclass,
        sex,
        COUNT(*) AS passenger_count,
        ROUND(AVG(CAST(survived AS DOUBLE)) * 100, 2) AS survival_rate_pct
    FROM titanic
    GROUP BY pclass, sex
    ORDER BY pclass, sex;
    """
).df()
# Breaks down survival rates across class and sex combinations.


## A16 – Average Age by Survival and Class

_Determine the average age for survivors and non-survivors within each `pclass`, excluding rows with NULL `age`._

In [ ]:
con.execute(
    """
    SELECT
        pclass,
        survived,
        ROUND(AVG(age), 2) AS avg_age
    FROM titanic
    WHERE age IS NOT NULL
    GROUP BY pclass, survived
    ORDER BY pclass, survived;
    """
).df()
# Shows age differences by survival outcome within each class.


## A17 – Large Family Proportion

_Create a `family_size` metric as `sibsp + parch + 1` and compute the percentage of passengers with `family_size > 3`._

In [ ]:
con.execute(
    """
    WITH families AS (
        SELECT
            *,
            sibsp + parch + 1 AS family_size
        FROM titanic
    )
    SELECT
        ROUND(100.0 * SUM(CASE WHEN family_size > 3 THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_large_family
    FROM families;
    """
).df()
# Reveals how common larger travel parties were on the Titanic manifest.


## A18 – Survival by Embarkation Town

_Report the survival rate for each `embark_town`, along with the passenger count per town, ordered by survival rate descending._

In [ ]:
con.execute(
    """
    SELECT
        embark_town,
        COUNT(*) AS passenger_count,
        ROUND(AVG(CAST(survived AS DOUBLE)) * 100, 2) AS survival_rate_pct
    FROM titanic
    WHERE embark_town IS NOT NULL
    GROUP BY embark_town
    ORDER BY survival_rate_pct DESC;
    """
).df()
# Compares survival outcomes by embarkation location.


## A19 – Survival by Age Bucket

_Group passengers into age buckets (0-12, 13-17, 18-29, 30-49, 50+) and compute the survival rate in each bucket._

In [ ]:
con.execute(
    """
    WITH bucketed AS (
        SELECT
            CASE
                WHEN age < 13 THEN '0-12'
                WHEN age BETWEEN 13 AND 17 THEN '13-17'
                WHEN age BETWEEN 18 AND 29 THEN '18-29'
                WHEN age BETWEEN 30 AND 49 THEN '30-49'
                ELSE '50+'
            END AS age_bucket,
            survived
        FROM titanic
        WHERE age IS NOT NULL
    )
    SELECT
        age_bucket,
        COUNT(*) AS passenger_count,
        ROUND(AVG(CAST(survived AS DOUBLE)) * 100, 2) AS survival_rate_pct
    FROM bucketed
    GROUP BY age_bucket
    ORDER BY CASE age_bucket
        WHEN '0-12' THEN 1
        WHEN '13-17' THEN 2
        WHEN '18-29' THEN 3
        WHEN '30-49' THEN 4
        ELSE 5
    END;
    """
).df()
# Highlights survival variability across broad age groups.


## A20 – Fare and Age Correlation

_Calculate the correlation coefficient between `fare` and `age`, ignoring rows where either value is NULL._

In [ ]:
con.execute(
    """
    SELECT
        ROUND(CORR(fare, age), 4) AS fare_age_corr
    FROM titanic
    WHERE fare IS NOT NULL
      AND age IS NOT NULL;
    """
).df()
# Measures the linear relationship between fare paid and passenger age.
